In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math

In [ ]:
def sHalf(T, sigma):
    temp = -np.log(T) * 2 * (sigma ** 2)
    return np.round(np.sqrt(temp))

def calculate_filter_size(T, sigma):
    return 2*sHalf(T, sigma) + 1

In [ ]:
def MaskGeneration(T, sigma):
    N = calculate_filter_size(T, sigma)
    shalf = sHalf(T, sigma)
    x, y = np.meshgrid(range(-int(shalf), int(shalf) + 1), range(-int(shalf), int(shalf) + 1))
    return x, y

In [ ]:
def Gaussian(x,y, sigma):
    temp = ((x ** 2) + (y ** 2)) / (2 * (sigma ** 2))
    return (np.exp(-temp))

def calculate_gradient_X(x,y, sigma):
    temp = (x ** 2 + y ** 2) / (2 * sigma ** 2)
    return -((x * np.exp(-temp)) / sigma ** 2)

def calculate_gradient_Y(x,y, sigma):
    temp = (x ** 2 + y ** 2) / (2 * sigma ** 2)
    return -((y * np.exp(-temp)) / sigma ** 2)

In [ ]:
def pad(img, kernel):
    r, c = img.shape
    kr, kc = kernel.shape
    padded = np.zeros((r + kr,c + kc), dtype=img.dtype)
    insert = np.uint((kr)/2)
    padded[insert: insert + r, insert: insert + c] = img
    return padded
            
def smooth(img, kernel=None):
    if kernel is None:
        mask = np.array([[1,1,1],[1,1,1],[1,1,1]])
    else:
        mask = kernel
    i, j = mask.shape
    output = np.zeros((img.shape[0], img.shape[1]))           
    image_padded = pad(img, mask)
    for x in range(img.shape[0]):    
        for y in range(img.shape[1]):
            output[x, y] = (mask * image_padded[x:x+i, y:y+j]).sum() / mask.sum()  
    return output


In [ ]:
def Create_Gx(fx, fy):
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            fx[i, j] = calculate_gradient_X(x[i, j], y[i, j], sigma)
    gx = np.round((fx * 255)/fx.min())
    gx = np.array(gx)
    return gx

def Create_Gy(fx, fy):    
    for i in range(y.shape[0]):
        for j in range(y.shape[1]):
            fy[i, j] = calculate_gradient_Y(x[i, j], y[i, j], sigma)
    gy = np.round(fy * 255/fy.min())
    gy = np.array(gy)
    return gy

In [ ]:
def ApplyMask(image, kernel):
    i, j = kernel.shape
    kernel = np.flipud(np.fliplr(kernel))    
    output = np.zeros_like(image)           
    image_padded = pad(image, kernel)
    for x in range(image.shape[0]):    
        for y in range(image.shape[1]):
            output[x, y] = (kernel * image_padded[x:x+i, y:y+j]).sum()        
    return output

In [ ]:
def Gradient_Magnitude(fx, fy):
    mag = np.zeros((fx.shape[0], fx.shape[1]))
    mag = np.sqrt((fx ** 2) + (fy ** 2))
    mag *= 255 / mag.max()
    return mag

def Gradient_Direction(fx, fy):
    g_dir = np.zeros((fx.shape[0], fx.shape[1]))
    g_dir = np.rad2deg(np.arctan2(fx, fy)) + 180
    return g_dir * 255 / g_dir.max()

In [ ]:
def Digitize_angle(Angle):
    quantized = np.zeros((Angle.shape[0], Angle.shape[1]))
    for i in range(Angle.shape[0]):
        for j in range(Angle.shape[1]):
            if 0 <= Angle[i, j] <= 22.5 or 157.5 <= Angle[i, j] <= 202.5 or 337.5 < Angle[i, j] < 360:
                quantized[i, j] = 0
            elif 22.5 <= Angle[i, j] <= 67.5 or 202.5 <= Angle[i, j] <= 247.5:
                quantized[i, j] = 1
            elif 67.5 <= Angle[i, j] <= 122.5 or 247.5 <= Angle[i, j] <= 292.5:
                quantized[i, j] = 2
            elif 112.5 <= Angle[i, j] <= 157.5 or 292.5 <= Angle[i, j] <= 337.5:
                quantized[i, j] = 3
    return quantized
    
def Non_Max_Supp(quantized, mag):
    a, b = mag.shape
    quant = np.zeros((quantized.shape[0], quantized.shape[1]), np.uint8)
    for i in range(1, quantized.shape[0]-1):
        for j in range(1, quantized.shape[1]-1):   
            if quantized[i,j] == 0:
                if mag[i,j] >= mag[i, j+1] and mag[i,j] >= mag[i, j-1]:
                    quant[i, j] = mag[i, j]
                else:
                    quant[i, j] = 0 
            if quantized[i,j] == 1:
                if mag[i,j] >= mag[i-1, j+1] and mag[i,j] >= mag[i+1, j-1]:
                    quant[i, j] = mag[i, j]
                else:
                    quant[i, j] = 0
            if quantized[i,j] == 2:
                if mag[i,j] >= mag[i-1, j] and mag[i,j] >= mag[i+1, j]:
                    quant[i, j] = mag[i, j]
                else:
                    quant[i, j] = 0
            if quantized[i,j] == 3:
                if mag[i, j] >= mag[i-1,j-1] and mag[i, j] >= mag[i+1,j+1]:
                    quant[i, j] = mag[i, j]
                else:
                    quant[i, j] = 0
    return quant

In [ ]:
def _double_thresholding(g_suppressed, low_threshold, high_threshold):
    g_thresholded = np.zeros(g_suppressed.shape)
    for i in range(0, g_suppressed.shape[0]):		# loop over pixels
        for j in range(0, g_suppressed.shape[1]):
            if g_suppressed[i,j] < low_threshold:	# lower than low threshold
                g_thresholded[i,j] = 0
            elif g_suppressed[i,j] >= low_threshold and g_suppressed[i,j] < high_threshold: 	# between thresholds
                g_thresholded[i,j] = 128
            else:					        # higher than high threshold
                g_thresholded[i,j] = 255
    return g_thresholded

def _hysteresis(g_thresholded):
    g_strong = np.zeros(g_thresholded.shape)
    for i in range(0, g_thresholded.shape[0]):		# loop over pixels
        for j in range(0, g_thresholded.shape[1]):
            val = g_thresholded[i,j]
            if val == 128:			# check if weak edge connected to strong
                if g_thresholded[i-1,j] == 255 or g_thresholded[i+1,j] == 255 or g_thresholded[i-1,j-1] == 255 or g_thresholded[i+1,j-1] == 255 or g_thresholded[i-1,j+1] == 255 or g_thresholded[i+1,j+1] == 255 or g_thresholded[i,j-1] == 255 or g_thresholded[i,j+1] == 255:
                    g_strong[i,j] = 255		# replace weak edge as strong
            elif val == 255:
                g_strong[i,j] = 255		# strong edge remains as strong edge
    return g_strong

# Step 1 Specify sigma and T value Also calculate Gradient masks
Here you will give the values of sigma and T(0-1). This will create the size of filter automatically. We generate gradient masks in x and y directions i.e. Gx and Gy

In [ ]:
sigma = 1
T = 0.3
x, y = MaskGeneration(T, sigma)
gauss = Gaussian(x, y, sigma)

fx = np.zeros((x.shape[0], x.shape[1]))
fy = np.zeros((y.shape[0], y.shape[1]))

gx = Create_Gx(fx, fy)
gy = Create_Gy(fx, fy)

# Step 2 Reading and converting image into grayscale
Here we convert the image into grayscale image for easy processing and finding edges

In [ ]:
image = cv2.imread('mecca06.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
plt.figure(figsize = (8,8))
plt.imshow(gray, cmap='gray')
plt.show()

## Smoothing
Here we smooth the image to reduce the intensity in the pixels

In [ ]:
smooth_img = smooth(gray, gauss)
plt.figure(figsize = (8,8))
plt.imshow(smooth_img, cmap='gray')

# Step 3 Applying the Gradient masks
In this step we apply the gradient x and y masks on the image.

In [ ]:
fx = ApplyMask(smooth_img, gx)
plt.figure(figsize = (8,8))
plt.imshow(fx, cmap='gray')

In [ ]:
fy = ApplyMask(smooth_img, gy)
plt.figure(figsize = (8,8))
plt.imshow(fy, cmap='gray')

# Step 4 Gradient magnitude
In this step we calculate the gradient magnitude at every pixel location

In [ ]:
mag = Gradient_Magnitude(fx, fy)
mag = mag.astype(int)
plt.figure(figsize = (8,8))
plt.imshow(mag, cmap='gray')
print('max', mag.max())
print('min', mag.min())

# STEP 5 Gradient Direction
In this step we find direction of gradient at each pixel of the image.

In [ ]:
Angle = Gradient_Direction(fx, fy)
plt.figure(figsize = (5,5))
plt.imshow(Angle, cmap='gray')
print('max', Angle.max())
print('min', Angle.min())

# Step 6 Quantization of angles and Non-Max Suppression
In this step we quantize our angles into 4 groups 0, 1, 2, 3. Then we apply non-maximum suppression on it to make the edges thin

In [ ]:
quantized = Digitize_angle(Angle)
nms = Non_Max_Supp(quantized, mag)
plt.figure(figsize = (10,10))
plt.imshow(nms, cmap='gray')
print('max', nms.max())
print('min', nms.min())

# Step 7 Double Threshold and Hysteresis
In this step we apply double threshold Tl and Th to our non-maximum suppressed images. After that we apply Hysteresis algorithm to get resultant edges of the image.

In [ ]:
threshold = _double_thresholding(nms, 10, 20)
cv2.imwrite('double_thresholded.jpg', threshold )
plt.figure(figsize = (10,10))
plt.imshow(threshold, cmap='gray')

In [ ]:
hys = _hysteresis(threshold)
cv2.imwrite('Result.jpg', hys)
plt.figure(figsize = (10,10))
plt.imshow(hys, cmap='gray')